<a href="https://colab.research.google.com/github/ashesh-12/Advanced-Chatbot/blob/main/Chapter1_Demo1_flask_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!sudo apt-get remove python3-blinker

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages were automatically installed and are no longer required:
  distro-info-data gir1.2-packagekitglib-1.0 libappstream4
  libpackagekit-glib2-18 libpolkit-agent-1-0 libpolkit-gobject-1-0
  libstemmer0d libxmlb2 libyaml-0-2 lsb-release packagekit pkexec policykit-1
  polkitd python-apt-common python3-apt python3-cffi-backend
  python3-cryptography python3-dbus python3-distro python3-gi python3-httplib2
  python3-jeepney python3-jwt python3-keyring python3-lazr.uri
  python3-pkg-resources python3-pyparsing python3-secretstorage python3-six
  python3-wadllib
Use 'sudo apt autoremove' to remove them.
The following packages will be REMOVED:
  python3-blinker python3-launchpadlib python3-lazr.restfulclient
  python3-oauthlib python3-software-properties software-properties-common
0 upgraded, 0 newly installed, 6 to remove and 0 not upgraded.
After this operation, 3,022 kB disk s

In [7]:
!pip install flask blinker

  Using cached flask-3.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
Using cached flask-3.1.0-py3-none-any.whl (102 kB)
Using cached blinker-1.9.0-py3-none-any.whl (8.5 kB)


In [8]:
!pip install flask transformers requests

In [56]:
!pip install transformers==4.41.2 # Replace with an appropriate version

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 99.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3


In [1]:
%%writefile flask_chatbot_app.py
from flask import Flask, request, jsonify
from transformers import pipeline

app = Flask(__name__)

# Load the Hugging Face model
try:
    chatbot_model = pipeline("text-generation", model="microsoft/Phi-3-mini-4k-instruct", trust_remote_code=True)
except Exception as e:
    print(f"Error loading model: {e}")
    # You might want to handle this error more gracefully in a production environment

@app.route('/chatbot', methods=['POST'])
def chatbot():
    user_input = request.json.get('message')
    if not user_input:
        return jsonify({'response': 'Please provide a message.'}), 400
    try:
        response = generate_response(user_input)
        return jsonify({'response': response})
    except Exception as e:
        print(f"Error generating response: {e}")
        return jsonify({'error': 'An error occurred while generating a response.'}), 500

def generate_response(user_input):
    # Generate a response using the Hugging Face model
    messages = [
        {"role": "user", "content": user_input},
    ]
    result = chatbot_model(messages, max_length=250, num_return_sequences=1)
    return result[0]['generated_text']

if __name__ == '__main__':
    app.run(debug=True, port=5000, host='0.0.0.0')

Overwriting flask_chatbot_app.py


In [8]:

import subprocess

# Stop any running Flask app
subprocess.run(['pkill', '-f', 'flask_chatbot_app.py'])

CompletedProcess(args=['pkill', '-f', 'flask_chatbot_app.py'], returncode=0)

In [3]:
!nohup python flask_chatbot_app.py &

nohup: appending output to 'nohup.out'


In [9]:
!sudo lsof -i -P -n | grep LISTEN

node          7 root   21u  IPv6 101521      0t0  TCP *:8080 (LISTEN)
kernel_ma    18 root    3u  IPv4 114101      0t0  TCP 172.28.0.12:6000 (LISTEN)
colab-fil   229 root    3u  IPv4  66089      0t0  TCP 127.0.0.1:3453 (LISTEN)
jupyter-n   258 root    7u  IPv4  78082      0t0  TCP 172.28.0.12:9000 (LISTEN)
python3   16134 root   28u  IPv4 450464      0t0  TCP 127.0.0.1:45771 (LISTEN)
python3   16214 root    3u  IPv4 496129      0t0  TCP 127.0.0.1:42537 (LISTEN)
python3   16214 root    5u  IPv4 496130      0t0  TCP 127.0.0.1:59441 (LISTEN)


In [6]:

import requests

# Define the URL of the Flask app
url = 'http://127.0.0.1:5000/chatbot'

# Send a request to the Flask app
response = requests.post(url, json={'message': 'Hello, how are you?'})
print(response.json())


{'response': [{'content': 'Hello, how are you?', 'role': 'user'}, {'content': " I'm doing well. How about you? How can I help you today?", 'role': 'assistant'}]}


In [7]:
response = requests.post(url, json={'message': 'I was wondering how can I go to Eiffel Tower from the airport using the train and subway?'})
print(response.json())

{'response': [{'content': 'I was wondering how can I go to Eiffel Tower from the airport using the train and subway?', 'role': 'user'}, {'content': ' To travel from Charles de Gaulle Airport to the Eiffel Tower using the train and subway, follow these steps:\n\n\n1. **Take the RER B Train**:\n\n   - Upon exiting the airport, head to the RER B train station, which is called "Aéroport Charles de Gaulle 2 TGV."\n\n   - Purchase a ticket for the RER B train. You can buy a single ticket or a Navigo Découverte pass for unlimited travel within a certain period.\n\n   - Board the RER B train towards "Châtelet - Les Halles."\n\n   - Get off at the "Châtelet - Les Halles" station.\n\n\n2. **Transfer to the Metro**:\n\n   - At "Châtelet - Les Halles," transfer to the Metro line 4.\n\n   - Exit the RER B station and enter the Metro station.\n\n   - Take the Metro line 4 towards "Mairie de Montrouge', 'role': 'assistant'}]}
